In [6]:
import csv
import pickle
from datetime import datetime

In [7]:
def load_csv(*files):
    """Загрузка таблицы из одного или нескольких CSV-файлов"""
    table = None
    for file in files:
        with open(file, mode='r', encoding='utf-8') as f:
            reader = csv.reader(f)
            data = list(reader)
            if table is None:
                table = data
            elif len(data[0]) != len(table[0]):
                raise ValueError(f"Несовпадение структуры таблиц в файле {file}")
            else:
                table.extend(data[1:])
    return table

def save_csv(table, file, max_rows=None):
    """Сохранение таблицы в CSV-файл. Поддержка разбивки на несколько файлов"""
    if max_rows and max_rows < len(table):
        for i in range(0, len(table), max_rows):
            chunk = table[i:i + max_rows]
            chunk_file = f"{file}_part_{i // max_rows + 1}.csv"
            with open(chunk_file, mode='w', encoding='utf-8', newline='') as f:
                writer = csv.writer(f)
                writer.writerows(chunk)
    else:
        with open(file, mode='w', encoding='utf-8', newline='') as f:
            writer = csv.writer(f)
            writer.writerows(table)

def load_pickle(*files):
    """Загрузка таблицы из одного или нескольких Pickle-файлов"""
    table = None
    for file in files:
        with open(file, mode='rb') as f:
            data = pickle.load(f)
            if table is None:
                table = data
            elif len(data[0]) != len(table[0]):
                raise ValueError(f"Несовпадение структуры таблиц в файле {file}")
            else:
                table.extend(data[1:])
    return table

def save_pickle(table, file, max_rows=None):
    """Сохранение таблицы в Pickle-файл. Поддержка разбивки на несколько файлов"""
    if max_rows and max_rows < len(table):
        for i in range(0, len(table), max_rows):
            chunk = table[i:i + max_rows]
            chunk_file = f"{file}_part_{i // max_rows + 1}.pkl"
            with open(chunk_file, mode='wb') as f:
                pickle.dump(chunk, f)
    else:
        with open(file, mode='wb') as f:
            pickle.dump(table, f)

In [8]:
class Table:
    def __init__(self, data):
        if not isinstance(data, list) or not all(isinstance(row, list) for row in data):
            raise TypeError("Данные должны представлять собой список списков")
        self.data = data
        self.header = data[0] if len(data) > 0 else []
        self.types = {col: str for col in range(len(self.header))}


    def get_rows_by_number(self, start, stop=None, copy_table=False):
        """Получение строк по номеру"""
        rows = self.data[start + 1: stop + 1]
        return Table([self.header] + rows) if copy_table else rows


    def get_rows_by_index(self, *indices, copy_table=False):
        """Получение строк с указанными значениями в первом столбце"""
        rows = [row for row in self.data if row[0] in indices]
        return Table([self.header] + rows) if copy_table else rows


    def get_column_types(self, by_number=True):
        """Получение типов данных в столбцах"""
        result = {}
        for i, col in enumerate(self.header):
            result[i if by_number else col] = self.types.get(i, str)
        return result


    def set_column_types(self, types_dict, by_number=True):
        """Установка типов данных для столбцов"""
        for key, value in types_dict.items():
            col_index = key if by_number else self.header.index(key)
            self.types[col_index] = value


    def get_values(self, column=0):
        """Получение значений из указанного столбца"""
        col_index = column if isinstance(column, int) else self.header.index(column)
        return [row[col_index] for row in self.data[1:]]


    def get_value(self, column=0):
        """Получение значения из столбца для таблицы с одной строкой"""
        if len(self.data) != 2:
            raise ValueError("Таблица должна содержать ровно одну строку")
        col_index = column if isinstance(column, int) else self.header.index(column)
        return self.data[1][col_index]


    def set_values(self, values, column=0):
        """Установка значений для столбца"""
        col_index = column if isinstance(column, int) else self.header.index(column)
        if len(values) != len(self.data) - 1:
            raise ValueError("Длина значений должна соответствовать количеству строк в таблице")
        for i, value in enumerate(values, start=1):
            self.data[i][col_index] = value


    def set_value(self, value, column=0):
        """Установка значения для столбца в таблице с одной строкой"""
        if len(self.data) != 2:
            raise ValueError("Таблица должна содержать ровно одну строку")
        col_index = column if isinstance(column, int) else self.header.index(column)
        self.data[1][col_index] = value


    def print_table(self):
        """Вывод таблицы на печать"""
        col_widths = [max(len(str(item)) for item in col) for col in zip(*self.data)]
        for row in self.data:
            print(" | ".join(str(item).ljust(width) for item, width in zip(row, col_widths)))


    def concat(self, other_table):
        """Склеивание двух таблиц"""
        if self.header != other_table.header:
            raise ValueError("Заголовки таблиц не совпадают")
        self.data.extend(other_table.data[1:])


    def split(self, row_number):
        """Разбиение таблицы на две по номеру строки"""
        if row_number < 1 or row_number >= len(self.data):
            raise ValueError("Номер строки вне диапазона")
        table1 = Table(self.data[:row_number + 1])
        table2 = Table([self.header] + self.data[row_number + 1:])
        return table1, table2


    def detect_column_types(self):
        """Определение типов данных по значениям в столбцах"""
        def infer_type(value):
            for dtype in (int, float, lambda x: datetime.strptime(x, "%Y-%m-%d"), str):
                try:
                    dtype(value)
                    return dtype
                except ValueError:
                    continue
            return str

        for col_index in range(len(self.header)):
            column_values = self.get_values(col_index)
            if not column_values:
                self.types[col_index] = str
            else:
                self.types[col_index] = infer_type(column_values[0])


    def add(self, column1, column2, result_column):
        """Сложение двух столбцов"""
        values1 = self.get_values(column1)
        values2 = self.get_values(column2)
        if len(values1) != len(values2):
            raise ValueError("Столбцы должны иметь одинаковую длину")
        result_values = [a + b for a, b in zip(values1, values2)]
        self.set_values(result_values, result_column)


    def sub(self, column1, column2, result_column):
        """Вычитание столбцов"""
        values1 = self.get_values(column1)
        values2 = self.get_values(column2)
        if len(values1) != len(values2):
            raise ValueError("Столбцы должны иметь одинаковую длину")
        result_values = [a - b for a, b in zip(values1, values2)]
        self.set_values(result_values, result_column)


    def mul(self, column1, column2, result_column):
        """Умножение столбцов"""
        values1 = self.get_values(column1)
        values2 = self.get_values(column2)
        if len(values1) != len(values2):
            raise ValueError("Столбцы должны иметь одинаковую длину")
        result_values = [a * b for a, b in zip(values1, values2)]
        self.set_values(result_values, result_column)


    def div(self, column1, column2, result_column):
        """Деление столбцов"""
        values1 = self.get_values(column1)
        values2 = self.get_values(column2)
        if len(values1) != len(values2):
            raise ValueError("Столбцы должны иметь одинаковую длину")
        result_values = [a / b if b != 0 else None for a, b in zip(values1, values2)]
        self.set_values(result_values, result_column)


In [9]:
# загружаем из CSV данные
table = load_csv('data1.csv', 'data2.csv')
print("Таблица из CSV:")
table_obj = Table(table)
table_obj.print_table()

# сохраняем в Pickle
save_pickle(table, 'data.pkl', max_rows=100)

# пример операции с таблицей(берём строки по номеру)
rows = table_obj.get_rows_by_number(0, 2, copy_table=True)
print("\nВыбранные строки:")
rows.print_table()


Таблица из CSV:
ID | Name    | Age
1  | Alice   | 25 
2  | Bob     | 30 
3  | Charlie | 35 
4  | David   | 40 
5  | Eve     | 28 
6  | Frank   | 50 

Выбранные строки:
ID | Name  | Age
1  | Alice | 25 
2  | Bob   | 30 
